<a href="https://colab.research.google.com/github/fedcba499/gslide/blob/main/gslide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.RUUNcI7F4E/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.nKR9Y8wVfW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.xJe2fWAAio/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://deb.debian.org/debian buster-updates InRelease
Get:5 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InReleas

In [ ]:
import os, time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from PIL import Image
from google.colab import files

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def capture_gslide(gslide_url):
    driver = web_driver()
    driver.implicitly_wait(10)
    driver.get(gslide_url)
    time.sleep(3)

    title = driver.title.split('.',1)[0]

    # get the number of slides
    actions = ActionChains(driver)
    actions.send_keys(Keys.END)
    actions.perform()
    max_page = int(driver.current_url.rsplit('.',1)[-1][1:])
    print("Total Number of Slides: " + str(max_page))
    time.sleep(3)

    # start the presentation
    keys = Keys()
    actions = ActionChains(driver)
    actions.send_keys(keys.CONTROL + keys.SHIFT + keys.F5)
    actions.key_down(keys.SHIFT).key_down(keys.CONTROL).send_keys(keys.F5)
    actions.perform()
    time.sleep(8)

    # move to next page until the end
    actions = ActionChains(driver)
    counter = 1
    image_list = []
    save_folder = title

    if not os.path.exists(save_folder):
      os.mkdir(save_folder)

    while counter <= int(max_page):
        if counter > int(driver.current_url.rsplit('.',1)[-1][1:]):
            counter -= 1
        fname = '{}.png'.format(str(counter).zfill(2))
        pdfname = title + ".pdf"
        save_dir = os.path.join(save_folder, fname)
        save_pdf = os.path.join(save_folder, pdfname)
        driver.save_screenshot(save_dir)
        # print('Page '+str(counter) + ' was captured.')

        im = Image.open(save_dir)
        image = im.convert('RGB')
        image_list.append(image)

        if(counter == max_page):
            image_list[0].save(save_pdf, save_all=True, append_images=image_list[1:])

        counter = int(driver.current_url.rsplit('.',1)[-1][1:])+1

        actions.click()
        actions.perform()

        time.sleep(2)


    print('Done')

    files.download(title+"/"+title+".pdf")

url = input("Please enter Google Slides Link - ")

capture_gslide(url)

Please enter Google Slides Link - https://docs.google.com/presentation/d/1Vdv3ifbKLDMWsAc1LzDyaaHl3dGP0oCA/edit?usp=drive_link
Total Number of Slides: 9
Done


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>